In [6]:
import os, sys
import numpy as np
import pandas as pd

## Obtain ROI List

In [7]:
cell_fea_dir = "../BatchCorrection/NoControl/CellFeas"
study_roi_list = sorted([os.path.splitext(ele)[0] for ele in os.listdir(cell_fea_dir) if ele.endswith(".npy")])

## Obtain Batch ID

In [8]:
study_slide_info_path = "../Metadata/StudySlide_Info.xlsx"
slide_slide_info = pd.read_excel(study_slide_info_path, sheet_name = "Sheet1", header= 0, index_col=None)

In [9]:
slide_batch_dict = {key: value for (key, value) in zip(slide_slide_info["Slide_ID"], slide_slide_info["Staining_ID"])}

In [10]:
roi_stain_list = [slide_batch_dict[ele[:ele.rfind("-")]] for ele in study_roi_list]

## Obtain ROI Location

In [11]:
study_roi_info_path = "../Metadata/StudyROI_Info.xlsx"
slide_roi_info = pd.read_excel(study_roi_info_path, sheet_name = "Sheet1", header= 0, index_col=None)

In [12]:
name_loc_dict = {key: value for (key, value) in zip(slide_roi_info["ROI_ID"], slide_roi_info["ROI_Location"])}
name_diag_dict = {key: value for (key, value) in zip(slide_roi_info["ROI_ID"], slide_roi_info["ROI_Diag"])}

In [13]:
roi_loc_list = []
for ele in study_roi_list:
    roi_loc = name_loc_dict[ele]
    if roi_loc in ["AdjacentNormal", "DistantNormal", "Normal"]:
        roi_loc_list.append(name_loc_dict[ele])
    elif roi_loc == "Tumor":
        roi_loc_list.append("Tumor" + "-" + name_diag_dict[ele])
    else:
        print("Unknow location: {}".format(roi_loc))

In [14]:
(unique, counts) = np.unique(roi_loc_list, return_counts=True)
{x:y for x,y in zip(unique, counts)}

{'AdjacentNormal': 94,
 'DistantNormal': 67,
 'Normal': 50,
 'Tumor-AAH': 226,
 'Tumor-ADC': 563,
 'Tumor-AIS': 314,
 'Tumor-MIA': 309}

## Save No Control Meta Information

In [15]:
nocontrol_meta_df = pd.DataFrame({
    "Filename": study_roi_list,
    "Batch": roi_stain_list,
    "Condition": roi_loc_list
})

In [16]:
nocontrol_meta_path = os.path.join("../BatchCorrection/Metadata", "NoControlMetaIMC.csv")
nocontrol_meta_df.to_csv(nocontrol_meta_path, index=False)